# Sources and Measurements Example

This example shows how to upload measurements from an external repository into the MultiViz Analytics Engine (MVG) service and how to read this data.

In [1]:
import json
import os
import sys
import logging
from pathlib import Path

import pandas as pd
import requests
from requests import HTTPError

# import mvg library with python bindings to mvg-API
from mvg import MVG

Note that the `TOKEN` is used both for authorization and authentication.
Thus, each unique token represents a unique user and each user has their own unique database on the VA service.

**You need to insert your token received from Viking Analytics here:**

In [2]:
# Replace by your own Token
VALID_TOKEN = os.environ['TEST_TOKEN']

This is Viking Analytics default logging setup which can be adapted to suit your needs.
Log messages are printed from `mvg` library, see the source code for details.

In [3]:
root_logger = logging.getLogger()
root_logger.setLevel("INFO")
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(name)s - %(message)s")
stream_handler = logging.StreamHandler(stream=sys.stderr)
stream_handler.setFormatter(formatter)
root_logger.addHandler(stream_handler)

## Downloading the Data

We pick the data from one source from our public charlie repo https://github.com/vikinganalytics/va-data-charlie.git for convenience.
Clone that repository to get access to the data.

In [4]:
!git clone https://github.com/vikinganalytics/va-data-charlie.git

fatal: destination path 'va-data-charlie' already exists and is not an empty directory.


We are going to use six of the sources from the charlie dataset with IDs `u0001` to `u0006`.

In [5]:
REF_DB_PATH = Path.cwd() / "va-data-charlie" / "charlieDb" / "acc"
SOURCE_IDS = ["u0001", "u0002", "u0003", "u0004", "u0005", "u0006"]

## Connect to the API

Instantiate a session object with mvg library a session object basically caches the endpoint and the token,
to simplify the calls to the MVG library.

In [6]:
ENDPOINT = "https://api.beta.multiviz.com"
session = MVG(ENDPOINT, VALID_TOKEN)

We now check if the server is alive. The hello message contains, amongst others the API version.

In [7]:
hello_message = json.dumps(session.say_hello())
print(hello_message)

2021-09-22 20:11:08,132 - INFO - mvg.mvg - Getting API info for: https://api.beta.multiviz.com


{"api": {"name": "MultiViz Engine API", "version": "v0.2.0", "swagger": "http://api.beta.multiviz.com/docs"}}


## Check Database

We start by seeing if there are any sources in the database.
A source represents a measurement source e.g. one specific channel of a measurement sensor.
Note that sources cannot be used to represent a sensor with several channels.
If this is required, the client side needs to take care of it.

Because we want to start at a clean slate we will list all sources (potentially none) and delete all of them

In [8]:
sources = session.list_sources()

print("Retrieved sources")
for src in sources:
    print(src)
    # While the list returned contains all information
    # about all known sources, it is also possible
    # to query for a single source by its id
    s_info = session.get_source(src['source_id'])
    print(f"Source info retrieved for one source: {s_info}")

2021-09-22 20:11:08,315 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:11:08,318 - INFO - mvg.mvg - listing sources
2021-09-22 20:11:08,490 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:11:08,491 - INFO - mvg.mvg - retrieving source with source id=u0001


Retrieved sources
{'source_id': 'u0001', 'meta': {'assetId': 'assetA', 'measPoint': 'mloc01', 'location': 'paris', 'updated': 'YES! I have been updated'}, 'properties': {'data_class': 'waveform', 'channels': ['acc']}}
Source info retrieved for one source: {'source_id': 'u0001', 'meta': {'assetId': 'assetA', 'measPoint': 'mloc01', 'location': 'paris', 'updated': 'YES! I have been updated'}, 'properties': {'data_class': 'waveform', 'channels': ['acc']}}


The example below revolves around a source with source id u0001

In [9]:
SOURCE_ID = SOURCE_IDS[0]

To make sure we start from a clean slate we delete our resource in case it exists.

In [10]:
try:
    source = session.get_source(SOURCE_ID)
    print(f"Deleting {SOURCE_ID}")
    session.delete_source(SOURCE_ID)
except HTTPError:
    print(f"Source {SOURCE_ID} does not exist")

2021-09-22 20:11:08,690 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:11:08,691 - INFO - mvg.mvg - retrieving source with source id=u0001
2021-09-22 20:11:08,881 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:11:08,881 - INFO - mvg.mvg - deleting source with source id=u0001


Deleting u0001


## Build measurements

Now we want to (re) build the source and the attached measurements from scratch

In [11]:
src_path = REF_DB_PATH / SOURCE_ID
m_file_name = REF_DB_PATH / SOURCE_ID / "meta.json"
with open(m_file_name, "r") as json_file:
    meta = json.load(json_file)
print("creating")
print(meta)

creating
{'assetId': 'assetA', 'measPoint': 'mloc01', 'location': 'paris'}


Create the source and check for it

In [12]:
session.create_source(SOURCE_ID, meta, channels=["acc"])  # create
source = session.get_source(SOURCE_ID)
print("Retrieved")
print(source)

2021-09-22 20:11:09,108 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:11:09,110 - INFO - mvg.mvg - creating source with source id=u0001
2021-09-22 20:11:09,111 - INFO - mvg.mvg - metadata: {'assetId': 'assetA', 'measPoint': 'mloc01', 'location': 'paris'}
2021-09-22 20:11:09,288 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:11:09,289 - INFO - mvg.mvg - retrieving source with source id=u0001


Retrieved
{'source_id': 'u0001', 'meta': {'assetId': 'assetA', 'measPoint': 'mloc01', 'location': 'paris'}, 'properties': {'data_class': 'waveform', 'channels': ['acc']}}


Update the source

In [13]:
meta['updated'] = "YES! I have been updated"
session.update_source(SOURCE_ID, meta)  # update
source = session.get_source(SOURCE_ID)
print("Retrieved")
print(source)

2021-09-22 20:11:09,473 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:11:09,475 - INFO - mvg.mvg - updating source with source id=u0001
2021-09-22 20:11:09,476 - INFO - mvg.mvg - metadata: {'assetId': 'assetA', 'measPoint': 'mloc01', 'location': 'paris', 'updated': 'YES! I have been updated'}
2021-09-22 20:11:09,654 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:11:09,655 - INFO - mvg.mvg - retrieving source with source id=u0001


Retrieved
{'source_id': 'u0001', 'meta': {'assetId': 'assetA', 'measPoint': 'mloc01', 'location': 'paris', 'updated': 'YES! I have been updated'}, 'properties': {'data_class': 'waveform', 'channels': ['acc']}}


Upload measurements to source. Measurements are tied to sources, they consist of

- an array of floating point values
- timestamp when the values were sampled
- a field for the duration of the measurement
- meta information to be stored along the measurement

In [14]:
meas = [f.stem for f in Path(src_path).glob("*.csv")]

`meas` now contains a list of timestamps representing the measurements in our repo we upload from iterate over all of them.

In [15]:
for m in meas:

    # samples file for one measurement
    TS_MEAS = str(m) + ".csv"  # filename
    TS_MEAS = REF_DB_PATH / SOURCE_ID / TS_MEAS  # path to file
    ts_df = pd.read_csv(TS_MEAS)  # read csv into df
    accs = ts_df.iloc[:, 0].tolist()  # convert to list
    print(f"Read {len(accs)} samples")

    # meta information file for one measurement
    TS_META = str(m) + ".json"  # filename
    TS_META = REF_DB_PATH / SOURCE_ID / TS_META  # path
    with open(TS_META, "r") as json_file:  # read json
        meas_info = json.load(json_file)  # into dict
    print(f"Read meta: {meas_info}")

    # get duration and other meta info
    duration = meas_info['duration']
    meta_info = meas_info['meta']
    # add sampling rate, not required by vibration API
    # but may be used on client side
    # in general any information can be stored
    # along the actual samples
    meta_info['sampling_rate'] = len(accs)/duration
    # <<< end of code specific for repo

    # Upload measurements
    print(f"Uploading {TS_MEAS}")
    try:
        # see mvg for details on this call
        session.create_measurement(sid=SOURCE_ID,
                                   duration=duration,
                                   timestamp=m,
                                   data={"acc": accs},
                                   meta=meta_info)
    except HTTPError as exc:
        print("OUCH")
        print(exc)

2021-09-22 20:11:09,897 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:11:09,898 - INFO - mvg.mvg - creating measurement from source id=u0001
2021-09-22 20:11:09,899 - INFO - mvg.mvg -   duration:  2.8672073400507907
2021-09-22 20:11:09,900 - INFO - mvg.mvg -   timestamp: 1570186860
2021-09-22 20:11:09,901 - INFO - mvg.mvg -   meta data: {'sampling_rate': 13950.857142857141}


Read 40000 samples
Read meta: {'duration': 2.8672073400507907, 'meta': {}}
Uploading ~/mvg/docs/source/content/examples/va-data-charlie/charlieDb/acc/u0001/1570186860.csv


2021-09-22 20:11:10,840 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:11:10,841 - INFO - mvg.mvg - creating measurement from source id=u0001
2021-09-22 20:11:10,842 - INFO - mvg.mvg -   duration:  2.8672073400507907
2021-09-22 20:11:10,842 - INFO - mvg.mvg -   timestamp: 1570273260
2021-09-22 20:11:10,843 - INFO - mvg.mvg -   meta data: {'sampling_rate': 13950.857142857141}


Read 40000 samples
Read meta: {'duration': 2.8672073400507907, 'meta': {}}
Uploading ~/mvg/docs/source/content/examples/va-data-charlie/charlieDb/acc/u0001/1570273260.csv


2021-09-22 20:11:11,835 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:11:11,836 - INFO - mvg.mvg - creating measurement from source id=u0001
2021-09-22 20:11:11,837 - INFO - mvg.mvg -   duration:  2.8672073400507907
2021-09-22 20:11:11,837 - INFO - mvg.mvg -   timestamp: 1570359660
2021-09-22 20:11:11,838 - INFO - mvg.mvg -   meta data: {'sampling_rate': 13950.857142857141}


...

Read 40000 samples
Read meta: {'duration': 2.8672073400507907, 'meta': {}}
Uploading ~/mvg/docs/source/content/examples/va-data-charlie/charlieDb/acc/u0001/1574251260.csv


2021-09-22 20:11:57,599 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:11:57,600 - INFO - mvg.mvg - creating measurement from source id=u0001
2021-09-22 20:11:57,600 - INFO - mvg.mvg -   duration:  2.8672073400507907
2021-09-22 20:11:57,601 - INFO - mvg.mvg -   timestamp: 1574337660
2021-09-22 20:11:57,601 - INFO - mvg.mvg -   meta data: {'sampling_rate': 13950.857142857141}


Read 40000 samples
Read meta: {'duration': 2.8672073400507907, 'meta': {}}
Uploading ~/mvg/docs/source/content/examples/va-data-charlie/charlieDb/acc/u0001/1574337660.csv


2021-09-22 20:11:59,114 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:11:59,115 - INFO - mvg.mvg - creating measurement from source id=u0001
2021-09-22 20:11:59,117 - INFO - mvg.mvg -   duration:  2.8672073400507907
2021-09-22 20:11:59,118 - INFO - mvg.mvg -   timestamp: 1574424060
2021-09-22 20:11:59,119 - INFO - mvg.mvg -   meta data: {'sampling_rate': 13950.857142857141}


Read 40000 samples
Read meta: {'duration': 2.8672073400507907, 'meta': {}}
Uploading ~/mvg/docs/source/content/examples/va-data-charlie/charlieDb/acc/u0001/1574424060.csv


## Read the Measurements

Check if we actually created the measurements by reading them

In [16]:
m = session.list_measurements(SOURCE_ID)
print(f"Read {len(m)} stored measurements")

2021-09-22 20:12:00,034 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:12:00,036 - INFO - mvg.mvg - retrieving all measurements from source id=u0001
2021-09-22 20:12:00,236 - INFO - mvg.mvg - 50 measurements in database


Read 50 stored measurements


It is also possible to read a specific measurement at a single timestamp.
Let's get the timestamp of the first measurement.

In [19]:
ts_0 = m[0]['timestamp']
meas_0 = session.read_single_measurement(SOURCE_ID, ts_0)
# we'll printout the returned measurement
print(f"source_id: {SOURCE_ID}")
print(f"timestamp: {ts_0}")
print(f"duration:  {meas_0['duration']}")
print(f"meta:      {meas_0['meta']}")
print(f"data:      {meas_0['data']['acc'][1:3]}...")

2021-09-22 20:13:04,777 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:13:04,779 - INFO - mvg.mvg - retrieving measurements from source id=u0001
2021-09-22 20:13:04,779 - INFO - mvg.mvg - timestamp=1570186860


source_id: u0001
timestamp: 1570186860
duration:  2.8672073400507907
meta:      {'sampling_rate': 13950.857142857141}
data:      [0.63897705078125, -0.55078125]...


We can update the meta information for a measurement at a single timestamp let's get the timestamp of the first measurement

In [21]:
ts_0 = m[0]['timestamp']
new_meta = meas_0['meta']
new_meta['updated'] = "YES!"
session.update_measurement(SOURCE_ID, ts_0, new_meta)
meas_0_u = session.read_single_measurement(SOURCE_ID, ts_0)

# we'll printout the returned measurement
print(f"source_id: {SOURCE_ID}")
print(f"timestamp: {ts_0}")
print(f"duration:  {meas_0_u['duration']}")
print(f"meta:      {meas_0_u['meta']}")
print(f"data:      {meas_0_u['data']['acc'][1:3]}...")

2021-09-22 20:13:29,345 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:13:29,346 - INFO - mvg.mvg - deleting measurement for source id=u0001
2021-09-22 20:13:29,347 - INFO - mvg.mvg -   timestamp: 1570186860
2021-09-22 20:13:29,531 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:13:29,532 - INFO - mvg.mvg - retrieving measurements from source id=u0001
2021-09-22 20:13:29,533 - INFO - mvg.mvg - timestamp=1570186860


source_id: u0001
timestamp: 1570186860
duration:  2.8672073400507907
meta:      {'sampling_rate': 13950.857142857141, 'updated': 'YES!'}
data:      [0.63897705078125, -0.55078125]...


Finally we delete the measurement

In [22]:
session.delete_measurement(SOURCE_ID, ts_0)

2021-09-22 20:13:34,597 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:13:34,599 - INFO - mvg.mvg - deleting measurement for source id=u0001
2021-09-22 20:13:34,599 - INFO - mvg.mvg -   timestamp: 1570186860


We check if it's actually deleted

In [23]:
try:
    meas_0 = session.read_single_measurement(SOURCE_ID, ts_0)
except HTTPError:
    print("Previously deleted measurement does not exist")

2021-09-22 20:13:38,561 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:13:38,563 - INFO - mvg.mvg - retrieving measurements from source id=u0001
2021-09-22 20:13:38,564 - INFO - mvg.mvg - timestamp=1570186860


Previously deleted measurement does not exist


If all went well we end up here now.
The source u0001 is in the database along with a number of its measurements.

## Uploading additional sources

Here we show how to upload additional sources in a more compact way.

In [26]:
for SOURCE_ID in SOURCE_IDS[1:]:

    NEW_SOURCE = SOURCE_ID

    try:
        print(f"Deleting {NEW_SOURCE}")
        session.delete_source(NEW_SOURCE)
    except HTTPError:
        print(f"Source {NEW_SOURCE} does not exist")
    
    src_path = REF_DB_PATH / NEW_SOURCE
    m_file_name = REF_DB_PATH / NEW_SOURCE / "meta.json"
    with open(m_file_name, "r") as json_file:
        meta = json.load(json_file)
    session.create_source(NEW_SOURCE, meta, channels=["acc"])  # create

    meas = [f.stem for f in Path(src_path).glob("*.csv")]

    for m in meas:

        # samples file for one measurement
        TS_MEAS = str(m) + ".csv"  # filename
        TS_MEAS = REF_DB_PATH / NEW_SOURCE / TS_MEAS  # path to file
        ts_df = pd.read_csv(TS_MEAS)  # read csv into df
        accs = ts_df.iloc[:, 0].tolist()  # convert to list
        print(f"Read {len(accs)} samples for {NEW_SOURCE}.")

        # meta information file for one measurement
        TS_META = str(m) + ".json"  # filename
        TS_META = REF_DB_PATH / NEW_SOURCE / TS_META  # path
        with open(TS_META, "r") as json_file:  # read json
            meas_info = json.load(json_file)  # into dict
        print(f"Read {NEW_SOURCE} meta: {meas_info}")

        duration = meas_info['duration']
        meta_info = meas_info['meta']
        meta_info['sampling_rate'] = len(accs)/duration

        print(f"Uploading {TS_MEAS}")
        try:
            # see mvg for details on this call
            session.create_measurement(sid=NEW_SOURCE,
                                       duration=duration,
                                       timestamp=m,
                                       data={"acc": accs},
                                       meta=meta_info)
        except HTTPError as exc:
            print(exc)




2021-09-22 20:52:39,692 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:52:39,696 - INFO - mvg.mvg - deleting source with source id=u0002


Deleting u0002


2021-09-22 20:52:39,956 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:52:39,958 - INFO - mvg.mvg - creating source with source id=u0002
2021-09-22 20:52:39,959 - INFO - mvg.mvg - metadata: {'assetId': 'assetB', 'measPoint': 'mloc01', 'location': 'paris'}
2021-09-22 20:52:40,179 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:52:40,180 - INFO - mvg.mvg - creating measurement from source id=u0002
2021-09-22 20:52:40,181 - INFO - mvg.mvg -   duration:  2.8672073400507907
2021-09-22 20:52:40,182 - INFO - mvg.mvg -   timestamp: 1570186860
2021-09-22 20:52:40,183 - INFO - mvg.mvg -   meta data: {'sampling_rate': 13950.857142857141}


Read 40000 samples for u0002.
Read u0002 meta: {'duration': 2.8672073400507907, 'meta': {}}
Uploading ~/mvg/docs/source/content/examples/va-data-charlie/charlieDb/acc/u0002/1570186860.csv


2021-09-22 20:52:41,154 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:52:41,155 - INFO - mvg.mvg - creating measurement from source id=u0002
2021-09-22 20:52:41,156 - INFO - mvg.mvg -   duration:  2.8672073400507907
2021-09-22 20:52:41,157 - INFO - mvg.mvg -   timestamp: 1570273260
2021-09-22 20:52:41,158 - INFO - mvg.mvg -   meta data: {'sampling_rate': 13950.857142857141}


...

Read 40000 samples for u0006.
Read u0006 meta: {'duration': 2.8672073400507907, 'meta': {}}
Uploading ~/mvg/docs/source/content/examples/va-data-charlie/charlieDb/acc/u0006/1574251260.csv


2021-09-22 20:56:43,111 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:56:43,112 - INFO - mvg.mvg - creating measurement from source id=u0006
2021-09-22 20:56:43,113 - INFO - mvg.mvg -   duration:  2.8672073400507907
2021-09-22 20:56:43,115 - INFO - mvg.mvg -   timestamp: 1574337660
2021-09-22 20:56:43,116 - INFO - mvg.mvg -   meta data: {'sampling_rate': 13950.857142857141}


Read 40000 samples for u0006.
Read u0006 meta: {'duration': 2.8672073400507907, 'meta': {}}
Uploading ~/mvg/docs/source/content/examples/va-data-charlie/charlieDb/acc/u0006/1574337660.csv


2021-09-22 20:56:44,057 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-09-22 20:56:44,058 - INFO - mvg.mvg - creating measurement from source id=u0006
2021-09-22 20:56:44,060 - INFO - mvg.mvg -   duration:  2.8672073400507907
2021-09-22 20:56:44,060 - INFO - mvg.mvg -   timestamp: 1574424060
2021-09-22 20:56:44,062 - INFO - mvg.mvg -   meta data: {'sampling_rate': 13950.857142857141}


Read 40000 samples for u0006.
Read u0006 meta: {'duration': 2.8672073400507907, 'meta': {}}
Uploading ~/mvg/docs/source/content/examples/va-data-charlie/charlieDb/acc/u0006/1574424060.csv
